# Walking Dinner Optimization Problem

A Walking Dinner is an event where two-person teams have starter, main dish and dessert at different locations in a city. Each team is hosting one course so that three teams meet for a course. The problem is to find the optimal team to team assignment for each course so that the way from course to the next is minimal but with several constraints.

The constraints are:
* no two teams should meet for more than one course
* every team hosts exactly one course
* teams located at the same address should meet

## Toy example

Let's create an example data set for this problem.

In [1]:
import numpy as np
from scipy.spatial import distance_matrix
import itertools

The number of teams should be a multiple since three teams meet for a course.

In [2]:
n_teams = 9

The coordinate for each team is drawn from a distribution.

In [3]:
x_coord = np.random.uniform(-10,10, n_teams)
y_coord = np.random.uniform(-10,10, n_teams)
team_locs = np.array([x_coord, y_coord]).T
team_locs[:5]

array([[ 5.8315653 , -2.07377013],
       [-8.43310776,  6.1047169 ],
       [-7.00695643,  2.16143881],
       [ 9.75944858,  2.04735725],
       [-5.58046728, -8.66195238]])

We calculate the pairwise distances between each team.

In [4]:
team_dist_matrix = np.sqrt(distance_matrix(team_locs, team_locs))
team_dist_matrix[:5, :5]

array([[0.        , 4.05498304, 3.67682569, 2.38603229, 3.6300415 ],
       [4.05498304, 0.        , 2.04774278, 4.3173497 , 3.87810308],
       [3.67682569, 2.04774278, 0.        , 4.09472748, 3.30408685],
       [2.38603229, 4.3173497 , 4.09472748, 0.        , 4.32531493],
       [3.6300415 , 3.87810308, 3.30408685, 4.32531493, 0.        ]])

We calculate the distance for every team between the three dishes

In [5]:
def assign_teams_to_course(n_teams):
    """Assign course to teams, so that the number of teams per course are equal
    
    Returns
    -------
    teams_per_course: np.array
        Array of shape (n_courses, n_teams_per_course)
    """
    teams = set(range(n_teams))
    n_teams_per_course = n_teams // 3
    n_courses = 3
    teams_per_course = []
    for course in range(n_courses):
        teams_in_course = np.random.choice(list(teams), size=n_teams_per_course, replace=False)
        teams.difference_update(teams_in_course)
        teams_per_course.append(teams_in_course)
    return np.array(teams_per_course)

In [6]:
host_teams = assign_teams_to_course(n_teams)
host_teams

array([[5, 7, 2],
       [0, 8, 3],
       [6, 4, 1]])

In [37]:
def create_random_distinct_triplets_with_host(n_teams, host_teams_for_course, not_allowed_match_for_teams=None):
    triplets = []
    guest_teams_pool = set(range(n_teams))
    guest_teams_pool.difference_update(host_teams_for_course)
    if not_allowed_match_for_teams is None:
        not_allowed_match_for_teams = [set() for _ in range(n_teams)]
        
    for host_team in host_teams_for_course:
        print(f"host_team: {host_team}")
        guest_teams = set()
        
        allowed_teams = guest_teams_pool.difference(not_allowed_match_for_teams[host_team])
        print(f"allowed_teams: {allowed_teams}")
        for i in range(100):
            guest_team = np.random.choice(list(allowed_teams), size=1)[0]
            print(f"test guest_team: {guest_team}")
            
            test_allowed_teams = allowed_teams.difference(not_allowed_match_for_teams[guest_team]).difference([guest_team])
            if len(test_allowed_teams) < 2-len(guest_teams)-1:
                print(f"host_team: {host_team}")
                print(f"test_allowed_teams: {test_allowed_teams}")
                print(f"not_allowed_match_for_teams[host_team]: {not_allowed_match_for_teams[host_team]}")
                print(f"guest_teams_pool: {guest_teams_pool}")
                print(f"triplets: {triplets}")
                print(f"discarding guest team {guest_team}")
                allowed_teams.remove(guest_team)
                if len(allowed_teams)==0:
                    print(f"host_team: {host_team}")
                    print(f"allowed_teams: {allowed_teams}")
                    print(f"not_allowed_match_for_teams[host_team]: {not_allowed_match_for_teams[host_team]}")
                    print(f"guest_teams_pool: {guest_teams_pool}")
                    print(f"triplets: {triplets}")
                    raise ValueError(f'Found no combination after {i} iterations')
                continue
            print(f"guest_team: {guest_team}")
            allowed_teams = test_allowed_teams
            guest_teams.add(guest_team)
            if len(guest_teams)==2:
                break

            if i==99:
                print(f"host_team: {host_team}")
                print(f"allowed_teams: {allowed_teams}")
                print(f"not_allowed_match_for_teams[host_team]: {not_allowed_match_for_teams[host_team]}")
                print(f"guest_teams_pool: {guest_teams_pool}")
                print(f"triplets: {triplets}")
                raise ValueError(f'Found no combination after {i} iterations')
        triplet = {host_team}
        triplet.update(guest_teams)
        guest_teams_pool.difference_update(triplet)
        for first_team_in_perm, second_team_in_perm in itertools.permutations(triplet, 2):
            not_allowed_match_for_teams[first_team_in_perm].add(second_team_in_perm)
        triplets.append(triplet)

    return triplets, not_allowed_match_for_teams

In [48]:

triplets_starter, not_allowed_match_for_teams = create_random_distinct_triplets_with_host(n_teams, host_teams[0])
print(f"triplets_starters: {triplets_starter}")
print(f"hosts_starter: {host_teams[0]}")
print(f"not_allowed_match_for_teams: {not_allowed_match_for_teams}")

triplets_main, not_allowed_match_for_teams = create_random_distinct_triplets_with_host(n_teams, host_teams[1], not_allowed_match_for_teams)
print(f"triplets_main: {triplets_main}")
print(f"hosts_main: {host_teams[1]}")
print(f"not_allowed_match_for_teams: {not_allowed_match_for_teams}")

triplets_dessert, not_allowed_match_for_teams = create_random_distinct_triplets_with_host(n_teams, host_teams[2], not_allowed_match_for_teams)
print(f"triplets_dessert: {triplets_dessert}")
print(f"hosts_dessert: {host_teams[2]}")
print(f"not_allowed_match_for_teams: {not_allowed_match_for_teams}")


host_team: 5
allowed_teams: {0, 1, 3, 4, 6, 8}
test guest_team: 1
guest_team: 1
test guest_team: 0
guest_team: 0
host_team: 7
allowed_teams: {8, 3, 4, 6}
test guest_team: 4
guest_team: 4
test guest_team: 8
guest_team: 8
host_team: 2
allowed_teams: {3, 6}
test guest_team: 6
guest_team: 6
test guest_team: 3
guest_team: 3
triplets_starters: [{0, 1, 5}, {8, 4, 7}, {2, 3, 6}]
hosts_starter: [5 7 2]
not_allowed_match_for_teams: [{1, 5}, {0, 5}, {3, 6}, {2, 6}, {8, 7}, {0, 1}, {2, 3}, {8, 4}, {4, 7}]
host_team: 0
allowed_teams: {2, 4, 6, 7}
test guest_team: 7
guest_team: 7
test guest_team: 2
guest_team: 2
host_team: 8
allowed_teams: {1, 5, 6}
test guest_team: 6
guest_team: 6
test guest_team: 1
guest_team: 1
host_team: 3
allowed_teams: {4, 5}
test guest_team: 4
guest_team: 4
test guest_team: 5
guest_team: 5
triplets_main: [{0, 2, 7}, {8, 1, 6}, {3, 4, 5}]
hosts_main: [0 8 3]
not_allowed_match_for_teams: [{1, 2, 5, 7}, {0, 8, 5, 6}, {0, 3, 6, 7}, {2, 4, 5, 6}, {8, 3, 5, 7}, {0, 1, 3, 4}, {8, 1,

ValueError: Found no combination after 1 iterations

In [9]:
n_teams // 3

3

It is possible, that the three teams in a triplet host the same course. 

Example for `n_teams=9`:
```
triplets_starters: [{1, 3, 5}, {0, 4, 7}, {8, 2, 6}]
hosts_starter: [3, 7, 6]
triplets_main: [{4, 5, 6}, {2, 3, 7}, {0, 8, 1}]
hosts_main: [4, 2, 0]
triplets_dessert: [{8, 3, 4}, {0, 2, 5}, {1, 6, 7}]
hosts_dessert: [8, 5, 1]
```
In this example e.g. in the first starter triplet `{1, 3, 5}` teams `1` and `5` host the dessert.

On the other hand, after the following pairings for the first two courses

```
triplets_starters: [{0, 1, 5}, {8, 4, 7}, {2, 3, 6}]
hosts_starter: [5, 7, 2]
triplets_main: [{0, 2, 7}, {8, 1, 6}, {3, 4, 5}]
hosts_main: [0, 8, 3]
```

and the first triplet `{5, 6, 7}` (host team 6) for the dessert, there is not allowed pairing for the dessert host 4 left. 4 has been paired with `{8, 3, 5, 7}` in the previous courses. 1 and 6 have to host the dessert for another triplet. The only left teams 2 and 0 have already met at the main dish. 